[View in Colaboratory](https://colab.research.google.com/github/JacksonIsaac/colab_notebooks/blob/master/cython_live_with_siraj.ipynb)

In [1]:
!pip install cython
!pip install spacy

    100% |████████████████████████████████| 3.4MB 4.3MB/s 
    100% |████████████████████████████████| 22.0MB 1.0MB/s 
    100% |████████████████████████████████| 122kB 21.1MB/s 
    100% |████████████████████████████████| 1.2MB 14.0MB/s 
    100% |████████████████████████████████| 194kB 23.6MB/s 
    100% |████████████████████████████████| 153kB 21.8MB/s 
    100% |████████████████████████████████| 604kB 16.7MB/s 
    100% |████████████████████████████████| 317kB 18.5MB/s 
    100% |████████████████████████████████| 450kB 21.5MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/60/0

In [0]:
from random import random

class Rectangle:
    def __init__(self, h, w):
        self.h = h
        self.w = w
        
    def area(self):
        return self.h * self.w
    
def check_rectangle_py(rectangles, threshold):
    n_out = 0
    
    for rectangle in rectangles:
        if rectangle.area() > threshold:
            n_out += 1
    return n_out

def main_rectangle_slow():
    n_rectangles = 10000000
    rectangles = list(Rectangle(random(), random()) for i in range(n_rectangles))
    n_out = check_rectangle_py(rectangles, threshold = 0.25)
    print(n_out)

In [3]:
%%time
main_rectangle_slow()

4035288
CPU times: user 16.8 s, sys: 1.88 s, total: 18.7 s
Wall time: 18.7 s


In [0]:
## Cython begins
%load_ext Cython

In [0]:
%%cython

from cymem.cymem cimport Pool
from random import random

cdef struct Rectangle:
    float w
    float h
    
cdef int check_rectangle_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    cdef int n_out = 0
    
    for rectangle in rectangles[:n_rectangles]:
        if rectangle.w * rectangle.h > threshold:
            n_out += 1
    return n_out

def main_rectangle_fast():
    cdef int n_rectangles = 10000000
    cdef float threshold = 0.25
    
    cdef Pool mem = Pool()
    
    cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    
    for i in range(n_rectangles):
        rectangles[i].w = random()
        rectangles[i].h = random()
    n_out = check_rectangle_cy(rectangles, n_rectangles, threshold)
    print(n_out)

In [14]:
%%time
main_rectangle_fast()

4033408
CPU times: user 768 ms, sys: 15.4 ms, total: 784 ms
Wall time: 788 ms


In [15]:
## Spacy begins

import spacy.cli

spacy.cli.download('en')
nlp = spacy.load('en')

multiSentence = 'Build it, train it, test it, makes it, denser, deeper, faster, smarter' \
                'I i i got to read papers, to try and make me smarter, I train my model' \
                'in the cloud now'

parsedData = nlp(multiSentence)


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [17]:
## POS Tagging

for span in parsedData.sents:
    sent = [parsedData[i] for i in range(span.start, span.end)]
    break

for token in sent:
    print(token.orth_, token.pos_)

Build VERB
it PRON
, PUNCT
train VERB
it PRON
, PUNCT
test VERB
it PRON
, PUNCT
makes VERB
it PRON
, PUNCT
denser NOUN
, PUNCT
deeper ADJ
, PUNCT
faster ADV
, PUNCT
smarterI PROPN


In [20]:
## dependency tagging

example = 'when they update the weights, they call it backpropagate, thats what'

parsedEx = nlp(example)

for token in parsedEx:
    print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts],
         [t.orth_ for t in token.rights])

when advmod update [] []
they nsubj update [] []
update advcl call ['when', 'they'] ['weights']
the det weights [] []
weights dobj update ['the'] []
, punct call [] []
they nsubj call [] []
call ccomp s ['update', ',', 'they'] ['it', 'backpropagate']
it dobj call [] []
backpropagate oprd call [] []
, punct s [] []
that nsubj s [] []
s ROOT s ['call', ',', 'that'] ['what']
what dep s [] []
